In [11]:
import pandas as pd
import numpy as np

train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }
        

data0 = pd.read_csv('../input/train.csv', header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes)
data = pd.read_csv('../input/test.csv', header=0,usecols=test_cols,parse_dates=["click_time"], dtype=dtypes)

print('len read:',len(data))

len read: 18790469


In [12]:
#attributed_data = data.query('is_attributed == 1')

#print('len of attributed:',len(attributed_data))

In [13]:
print(1.0 - 456846/184903890)

0.997529278589001


In [ ]:
print('sampling data')
#data = data.set_index('ip').loc[lambda x: (x.index + 401) % 10 == 0].reset_index()

data['hour'] = data["click_time"].dt.hour.astype('uint8')
data['day'] = data["click_time"].dt.day.astype('uint8')

data.drop('click_time', inplace=True, axis=1)

In [25]:
data0['hour'] = data0["click_time"].dt.hour.astype('uint8')
data0['day'] = data0["click_time"].dt.day.astype('uint8')

data0.drop('click_time', inplace=True, axis=1)

In [15]:
#data.drop('click_time', inplace=True, axis=1)

print('added hour and day')



added hour and day


In [26]:
#print(data.describe())

data0 = data0.query('hour in [{}]'.format(','.join(map(lambda x: str(x), set(data['hour'].unique())))))
print(data0['hour'].unique())

hour_set = set(data['hour'].unique())
hour_set0 = set(data0['hour'].unique())


[14 15  4  5  6  9 10 11 13]


In [27]:
osset = set(data['os'].unique())
osset0 = set(data0['os'].unique())

print(len(osset & osset0))
print(len(osset0))

315
573


In [28]:
chset = set(data['app'].unique())
chset0 = set(data0['app'].unique())

print(len(chset & chset0))
print(len(chset0))

384
610
